Aufgaben zu Vertiefung AI-Engineer Modul 5 - Time-Series-Data

Aufgabe 1: Time-Series-Data

Identifizieren Sie im UCI Repository (oder von anderen Stellen) einen Datensatz mit temporaler 
Dynamik. Implementieren Sie ein Neuronales Netz mit dem “naiven Ansatz”, mehrere Instanzen 
nachrutschend in die Input Schicht zu geben. Evaluieren Sie diesen naiven Ansatz gegen eine 
Implementierung mittels rekurrenter Layer.

https://archive.ics.uci.edu/datasets.php

Dow Jones Index
Donated on 10/22/2014

https://archive.ics.uci.edu/dataset/312/dow+jones+index

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics


import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
data = pd.read_csv("../data/data/dow_jones_index.csv")

In [ ]:
data ["close"] = data["close"] .apply(lambda x: x.replace("$",""))
data ["open"] = data["open"] .apply(lambda x: x.replace("$",""))
data ["high"] = data["high"] .apply(lambda x: x.replace("$",""))
data ["low"] = data["low"] .apply(lambda x: x.replace("$",""))
data ["next_weeks_open"] = data["next_weeks_open"] .apply(lambda x: x.replace("$",""))
data ["next_weeks_close"] = data["next_weeks_close"] .apply(lambda x: x.replace("$",""))


#df['DataFrame Column'] = df['DataFrame Column'].astype(float)

data ["close"] = data["close"].astype(float) 
data ["open"] = data["open"].astype(float)
data ["high"] = data["high"].astype(float)
data ["low"] = data["low"].astype(float)
data ["volume"] = data["volume"].astype(float)
data ["percent_change_price"] = data["percent_change_price"].astype(float)
data ["next_weeks_open"] = data["next_weeks_open"].astype(float)
data ["next_weeks_close"] = data["next_weeks_close"].astype(float)
data ["percent_change_next_weeks_price"] = data["percent_change_next_weeks_price"].astype(float)
data ["days_to_next_dividend"] = data["days_to_next_dividend"].astype(float)
data ["percent_return_next_dividend"] = data["percent_return_next_dividend"].astype(float)

data['close'].dtype

In [ ]:
data.shape

(750, 16)

In [ ]:
data.head(5)

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,1/7/2011,15.82,16.72,15.78,16.42,239655616.0,3.79267,NaN,NaN,16.71,15.97,-4.428490,26.0,0.182704
1,1,AA,1/14/2011,16.71,16.71,15.64,15.97,242963398.0,-4.42849,1.380223,239655616.0,16.19,15.79,-2.470660,19.0,0.187852
2,1,AA,1/21/2011,16.19,16.38,15.60,15.79,138428495.0,-2.47066,-43.024959,242963398.0,15.87,16.13,1.638310,12.0,0.189994
3,1,AA,1/28/2011,15.87,16.63,15.82,16.13,151379173.0,1.63831,9.355500,138428495.0,16.18,17.14,5.933250,5.0,0.185989
4,1,AA,2/4/2011,16.18,17.39,16.18,17.14,154387761.0,5.93325,1.987452,151379173.0,17.33,17.37,0.230814,97.0,0.175029


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd


x = data[['open', 'high', 'low', 'volume', 'percent_change_price', 'next_weeks_open', 'next_weeks_close', 'percent_change_next_weeks_price', 'days_to_next_dividend', 'percent_return_next_dividend']].to_numpy()
y = data["close"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)



In [ ]:
x.shape

(750, 10)

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32)

y_train = torch.tensor(y_train.values, dtype=torch.float32)

x_test = torch.tensor(x_test, dtype=torch.float32)

y_test = torch.tensor(y_test.values, dtype=torch.float32)




In [ ]:
print(type(x_train))
x_train.shape

<class 'torch.Tensor'>


torch.Size([502, 10])

In [ ]:
import numpy as np
import torch
import torch.nn as nn

# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define the LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        
        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        return out, (hidden, cell)

# Assuming you have defined device somewhere
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
input_size = 10
output_size = 1  # Assuming it's a regression problem
hidden_dim = 32
n_layers = 2
sequence_length = 20
batch_size = 8

# Hidden Dim = 16, Layers = 2, Learning Rate = 0.01, Batch Size = 2
# Hidden Dim = 64, Layers = 2, Learning Rate = 0.01, Batch Size = 2
# Create an instance of the LSTM model
model = LSTMModel(input_size, output_size, hidden_dim, n_layers).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()  # Use Mean Squared Error for regression

# Define learning rate
learning_rate = 0.01

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Generate random input and target data
x_train = np.random.rand(batch_size, sequence_length, input_size)
y_train = np.random.rand(batch_size, sequence_length, output_size) * 100  # Generating continuous target values

x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)

# Training loop
num_epochs = 2000

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs, _ = model(x_train)
    outputs = outputs.view(-1, output_size)

    # Calculate loss
    loss = loss_function(outputs, y_train.view(-1, output_size))

    # Backpropagation
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Code for making predictions
input_data = np.array([15.82, 16.72, 15.78, 239655616.0, 3.79267, 16.71, 15.97, -4.428490, 26.0, 0.182704])
input_data = input_data.reshape(1, -1, input_size)
input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    prediction_tuple = model(input_tensor)
predicted_value = prediction_tuple[0].item()

print("Predicted Value:", predicted_value)


Epoch [100/2000], Loss: 1195.2109
Epoch [200/2000], Loss: 896.3358
Epoch [300/2000], Loss: 880.3021
Epoch [400/2000], Loss: 880.0225
Epoch [500/2000], Loss: 880.0092
Epoch [600/2000], Loss: 880.0000
Epoch [700/2000], Loss: 879.9922
Epoch [800/2000], Loss: 879.9852
Epoch [900/2000], Loss: 879.9783
Epoch [1000/2000], Loss: 879.9715
Epoch [1100/2000], Loss: 879.9645
Epoch [1200/2000], Loss: 879.9573
Epoch [1300/2000], Loss: 879.9498
Epoch [1400/2000], Loss: 879.9419
Epoch [1500/2000], Loss: 879.9337
Epoch [1600/2000], Loss: 879.9250
Epoch [1700/2000], Loss: 879.9159
Epoch [1800/2000], Loss: 879.9064
Epoch [1900/2000], Loss: 879.8961
Epoch [2000/2000], Loss: 879.8854
Predicted Value: 38.22936248779297


In [ ]:
ö

In [ ]:

import itertools
import numpy as np
import torch

# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

"""
# Define hyperparameter grid
hidden_dim_values = [32]
n_layers_values = [2]
learning_rate_values = [0.01]
batch_size_values = [4, 8, 16]
"""""
# Define hyperparameter grid
hidden_dim_values = [16]
n_layers_values = [2]
learning_rate_values = [0.01]
batch_size_values = [2]

""""
# Define hyperparameter grid
hidden_dim_values = [2,4,8,16,32,64,128,512,1024,2048]
n_layers_values = [2,4,8,16,32,64,128,512,1024,2048]
learning_rate_values = [0,1, 0.01, 0.001, 0.0001, 0.00001]
batch_size_values = [1,2,3,4,5,6,7,8,9,10,11,13,14,15,16]
"""

# Create all possible combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(hidden_dim_values, n_layers_values, learning_rate_values, batch_size_values))

best_loss = float('inf')  # Initialize with a high value
best_parameters = None

# Training loop with grid search
for idx, (hidden_dim, n_layers, learning_rate, batch_size) in enumerate(hyperparameter_combinations):
    print(f"Training with Hyperparameters: Hidden Dim = {hidden_dim}, Layers = {n_layers}, Learning Rate = {learning_rate}, Batch Size = {batch_size}")

    # Create an instance of the LSTM model with the current hyperparameters
    model = LSTMModel(input_size, output_size, hidden_dim, n_layers).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Generate random input and target data for the current batch size
    x_train = np.random.rand(batch_size, sequence_length, input_size)
    y_train = np.random.rand(batch_size, sequence_length, output_size) * 100

    x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.float32).to(device)

    # Training loop
    num_epochs = 6000

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        outputs, _ = model(x_train)
        outputs = outputs.view(-1, output_size)

        # Calculate loss
        loss = loss_function(outputs, y_train.view(-1, output_size))

        # Backpropagation
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 400 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    #print(f"Finished Training with Hyperparameters: Hidden Dim = {hidden_dim}, Layers = {n_layers}, Learning Rate = {learning_rate}, Batch Size = {batch_size}")
    print (         )
    # Update the best parameters if this run has a lower loss
    if loss < best_loss:
        best_loss = loss
        best_parameters = (hidden_dim, n_layers, learning_rate, batch_size)

print("\nBest Parameters Combination:")
print(f"Hidden Dim = {best_parameters[0]}, Layers = {best_parameters[1]}, Learning Rate = {best_parameters[2]}, Batch Size = {best_parameters[3]}")


Training with Hyperparameters: Hidden Dim = 8, Layers = 2, Learning Rate = 0.01, Batch Size = 2


Epoch [400/6000], Loss: 1243.7919
Epoch [800/6000], Loss: 961.5900
Epoch [1200/6000], Loss: 947.6854
Epoch [1600/6000], Loss: 947.4916
Epoch [2000/6000], Loss: 947.4724
Epoch [2400/6000], Loss: 947.4508
Epoch [2800/6000], Loss: 926.8208
Epoch [3200/6000], Loss: 926.6218
Epoch [3600/6000], Loss: 926.4611
Epoch [4000/6000], Loss: 926.2775
Epoch [4400/6000], Loss: 926.0581
Epoch [4800/6000], Loss: 925.7931
Epoch [5200/6000], Loss: 925.4725
Epoch [5600/6000], Loss: 925.0854
Epoch [6000/6000], Loss: 924.6201

Training with Hyperparameters: Hidden Dim = 16, Layers = 2, Learning Rate = 0.01, Batch Size = 2
Epoch [400/6000], Loss: 864.3794
Epoch [800/6000], Loss: 835.9557
Epoch [1200/6000], Loss: 835.7936
Epoch [1600/6000], Loss: 835.6956
Epoch [2000/6000], Loss: 835.5961
Epoch [2400/6000], Loss: 835.4831
Epoch [2800/6000], Loss: 66.0741
Epoch [3200/6000], Loss: 3.1234
Epoch [3600/6000], Loss: 0.8146
Epoch [4000/6000], Loss: 0.4528
Epoch [4400/6000], Loss: 0.3143
Epoch [4800/6000], Loss: 0.221

llllllllllllll